In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.ops import unary_union
from shapely.geometry import Point, MultiPolygon, Polygon, LineString
import topojson as tp
import h3
import os
pd.set_option('display.max_columns', None)

In [6]:
def merge_time_columns(time_df):
    time_df['row_col'] = time_df.apply(lambda row: [row['row'], row['col']], axis=1)
    time_df['x_y'] = time_df.apply(lambda row: [row['x'], row['y']], axis=1)
    time_df['x_ras_y_ras'] = time_df.apply(lambda row: [row['x_ras'], row['y_ras']], axis=1)
    time_df.drop(['row', 'col', 'x', 'y', 'x_ras', 'y_ras'], axis=1, inplace=True)
    return time_df

def join_points_poly(points_df, poly_df, join_rules):
    merged = poly_df
    joined_df = gpd.sjoin(points_df, poly_df)
    grouped = joined_df.groupby('index_right').agg(join_rules)
    merged = merged.merge(grouped, left_index=True, right_index=True)
    return merged

# create a dictionary that is the  names of the column names in hex8 and the values are an empty list
def create_style_dictionary(df):
    style_hex_dict = {}
    for col in df.columns: 
        style_hex_dict[col] = { 'fill-color': [ "'interpolate', ['linear'], ['get', '"+col+"'], dataMap['"+col+"'][3]['style_stops_99'][0], palettes[dataMap['"+col+"'][2]][0], dataMap['"+col+"'][3]['style_stops_99'][1], palettes[dataMap['"+col+"'][2]][1], dataMap['"+col+"'][3]['style_stops_99'][2], palettes[dataMap['"+col+"'][2]][2], dataMap['"+col+"'][3]['style_stops_99'][3], palettes[dataMap['"+col+"'][2]][3], dataMap['"+col+"'][3]['style_stops_99'][4], palettes[dataMap['"+col+"'][2]][4]"]}
    return style_hex_dict

In [50]:
# Read in destination files as geodataframes
path = './unsynced-data/rwanda/travel-time'

# list the files in the path directory and if they end in .parquet, add them to a list called files
files = [file for file in os.listdir(path) if file.endswith('.parquet')]

datasets = []
for file in files:
    df = pd.read_parquet(path + '/' + file)
    # Convert the DataFrame to a GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['x'], df['y']))
    gdf.crs = 'EPSG:4326'
    datasets.append(gdf)

# Now datasets list contains GeoDataFrames

In [51]:
# doing some basic stuff with each dataframe
# for range of number of lists in datasets print the datasets number

for d in range(len(datasets)):
    print ("starting " +files[d]) 
    datasets[d] = datasets[d].replace([np.inf, -np.inf], None)
    datasets[d] = datasets[d].replace([np.nan], None)
    datasets[d] = merge_time_columns(datasets[d])
    # appending destination type to travel time and time delta columns
    datasets[d] = datasets[d].rename(columns={col: col + files[d][14:-8] for col in datasets[d].columns if col.startswith('travel_time')})
    datasets[d] = datasets[d].rename(columns={col: col + files[d][14:-8] for col in datasets[d].columns if col.startswith('time_delta')})
    
    datasets[d]['row_col'] = datasets[d]['row_col'].astype(str)
    print ("finished with " +files[d]) 

starting travel_time_to_secondary_schools_fixed.parquet
finished with travel_time_to_secondary_schools_fixed.parquet
starting travel_time_to_semi_dense_urban_optimal.parquet
finished with travel_time_to_semi_dense_urban_optimal.parquet
starting travel_time_to_major_hospitals_optimal.parquet
finished with travel_time_to_major_hospitals_optimal.parquet
starting travel_time_to_health_posts_optimal.parquet
finished with travel_time_to_health_posts_optimal.parquet
starting travel_time_to_health_centers_optimal.parquet
finished with travel_time_to_health_centers_optimal.parquet
starting travel_time_to_all_education_facilities_fixed.parquet
finished with travel_time_to_all_education_facilities_fixed.parquet
starting travel_time_to_primary_schools_fixed.parquet
finished with travel_time_to_primary_schools_fixed.parquet
starting travel_time_to_all_health_facilities_optimal.parquet
finished with travel_time_to_all_health_facilities_optimal.parquet


In [52]:
# merging all dataframes into one
# each dataframe brings it's unique travel times and travel deltas
for d in datasets:
    # I make the first df be the main one and then merge all of the rest into it.
    travel_time_merged = datasets[0]
    for d in range(1, len(datasets)):
        # make a list of values from columns that we're going to merge into the dataframe
        merge_col = []
        for c in list(datasets[d].columns):
            if c.startswith('travel_time') or c.startswith('time_delta'):
                merge_col.append(c)
        # merge the travel_time_merged df with the datasets[d] df on the row_col column
        travel_time_merged = pd.merge(travel_time_merged, datasets[d][['row_col'] + merge_col], on='row_col', how='left')
        # replace NaN values in travel_time_merged with values from datasets[d] wherever they exist
        travel_time_merged = datasets[d][['row_col'] + merge_col].set_index('row_col').combine_first(travel_time_merged.set_index('row_col')).reset_index()
        # fill NA values with values from datasets[d]
        travel_time_merged = travel_time_merged.fillna(datasets[d])


# convert travel_time_merged to geodataframe
travel_time_merged = gpd.GeoDataFrame(travel_time_merged, geometry='geometry')
travel_time_merged.crs = "EPSG:4326"

# drop x_ras_y_ras column
travel_time_merged = travel_time_merged.drop(['x_ras_y_ras'], axis=1)

In [53]:
travel_time_merged

,row_col,births,female_educational_attainment_mean,females_0_4,females_0_9,females_10_14,females_15_49,females_50_64,females_5_9,females_65_plus,geometry,male_educational_attainment_mean,males_0_4,males_0_9,males_10_14,males_15_49,males_50_64,males_5_9,males_65_plus,pop_0_4,pop_0_9,pop_10_14,pop_15_49,pop_50_64,pop_5_9,pop_65_plus,population,pregnancies,rwi,time_delta_constructed_sites_all_education_facilities_fixed,time_delta_constructed_sites_all_health_facilities_optimal,time_delta_constructed_sites_health_centers_optimal,time_delta_constructed_sites_health_posts_optimal,time_delta_constructed_sites_major_hospitals_optimal,time_delta_constructed_sites_primary_schools_fixed,time_delta_constructed_sites_secondary_schools_fixed,time_delta_constructed_sites_semi_dense_urban_optimal,time_delta_no_sites_all_education_facilities_fixed,time_delta_no_sites_all_health_facilities_optimal,time_delta_no_sites_health_centers_optimal,time_delta_no_sites_health_posts_optimal,time_delta_no_sites_major_hospitals_optimal,time_delta_no_sites_primary_schools_fixed,time_delta_no_sites_secondary_schools_fixed,time_delta_no_sites_semi_dense_urban_optimal,travel_time_all_education_facilities_fixed,travel_time_all_health_facilities_optimal,travel_time_constructed_sites_all_education_facilities_fixed,travel_time_constructed_sites_all_health_facilities_optimal,travel_time_constructed_sites_health_centers_optimal,travel_time_constructed_sites_health_posts_optimal,travel_time_constructed_sites_major_hospitals_optimal,travel_time_constructed_sites_primary_schools_fixed,travel_time_constructed_sites_secondary_schools_fixed,travel_time_constructed_sites_semi_dense_urban_optimal,travel_time_health_centers_optimal,travel_time_health_posts_optimal,travel_time_major_hospitals_optimal,travel_time_no_sites_all_education_facilities_fixed,travel_time_no_sites_all_health_facilities_optimal,travel_time_no_sites_health_centers_optimal,travel_time_no_sites_health_posts_optimal,travel_time_no_sites_major_hospitals_optimal,travel_time_no_sites_primary_schools_fixed,travel_time_no_sites_secondary_schools_fixed,travel_time_no_sites_semi_dense_urban_optimal,travel_time_primary_schools_fixed,travel_time_secondary_schools_fixed,travel_time_semi_dense_urban_optimal,underweight,x_y
0,"[10, 1897]",0.2158,4.724257,0.532922,1.102213,0.570825,1.990846,0.334852,0.569291,0.116536,POINT (30.43849 -1.05409),5.613448,0.523792,1.12397,0.606819,2.048758,0.421264,0.600178,0.120037,1.056714,2.226183,1.177644,4.039604,0.756116,1.169469,0.236573,8.436119,0.3056,-0.088528,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,81.0,82.0,81.0,82.0,82.0,253.0,870.0,81.0,81.0,507.0,82.0,253.0,870.0,81.0,82.0,82.0,253.0,870.0,81.0,81.0,507.0,81.0,81.0,507.0,0.09251,"[30.4384884, -1.0540862]"
1,"[10, 1898]",0.203684,4.725142,0.503002,1.04033,0.538776,1.879072,0.316052,0.537329,0.109993,POINT (30.43917 -1.05417),5.613625,0.494384,1.060866,0.57275,1.933732,0.397613,0.566482,0.113297,0.997386,2.101196,1.111526,3.812804,0.713665,1.10381,0.22329,7.962481,0.288443,-0.085817,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0,80.0,79.0,80.0,80.0,251.0,868.0,79.0,79.0,505.0,80.0,251.0,868.0,79.0,80.0,80.0,251.0,868.0,79.0,79.0,505.0,79.0,79.0,505.0,0.092532,"[30.439166560383864, -1.0541664609655237]"
2,"[10, 1899]",0.373898,4.731713,0.923348,1.909709,0.989018,3.449366,0.580169,0.986361,0.201912,POINT (30.44000 -1.05417),5.61857,0.907529,1.947405,1.051382,3.549704,0.729888,1.039876,0.207977,1.830877,3.857114,2.040401,6.999069,1.310057,2.026237,0.409889,14.616529,0.529487,-0.084257,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,77.0,78.0,77.0,78.0,78.0,249.0,866.0,77.0,77.0,503.0,78.0,249.0,866.0,77.0,78.0,78.0,249.0,866.0,77.0,77.0,503.0,77.0,77.0,503.0,0.092703,"[30.439999893713928, -1.0541664609655237]"
3,"[10, 1900]",0.579211,4.735256,1.430372,2.958358,1.532103,5.343463,0.898748,1.527986,0.312784,POINT (30.44113 -1.05446),5.620417,1.405866,3.016753,1.628711,5.49

In [55]:
travel_time_merged_proof = travel_time_merged.copy()

In [56]:
# change travel_time_merged['x_y'] to a string separated by a _
travel_time_merged['x_y'] = travel_time_merged['x_y'].astype(str).str.replace('[', '').str.replace(']', '').str.replace(' ', '').str.replace(',', '_')
# change row_col to a string separated by a _
travel_time_merged['row_col'] = travel_time_merged['row_col'].astype(str).str.replace('[', '').str.replace(']', '').str.replace(' ', '').str.replace(',', '_')

In [58]:
print(travel_time_merged['x_y'][0])
print(travel_time_merged['row_col'][0])

30.4384884_-1.0540862
10_1897


In [59]:
# make rwi equal to rwi times the population value in it's row
travel_time_merged['rwi'] = travel_time_merged['rwi'] * travel_time_merged['population']
travel_time_merged['underweight'] = travel_time_merged['underweight'] * travel_time_merged['population']
# travel_time_merged['male_educational_attainment_mean'] equals the sum the value of male_educational_attainment_mean times the sum of 'males_15_49'+'males_50_64'+'males_65_plus'
travel_time_merged['male_educational_attainment_mean'] = travel_time_merged['male_educational_attainment_mean'] * (travel_time_merged['males_15_49']+travel_time_merged['males_50_64']+travel_time_merged['males_65_plus'])
travel_time_merged['female_educational_attainment_mean'] = travel_time_merged['female_educational_attainment_mean'] * (travel_time_merged['females_15_49']+travel_time_merged['females_50_64']+travel_time_merged['females_65_plus'])


In [60]:
travel_time_merged

,row_col,births,female_educational_attainment_mean,females_0_4,females_0_9,females_10_14,females_15_49,females_50_64,females_5_9,females_65_plus,geometry,male_educational_attainment_mean,males_0_4,males_0_9,males_10_14,males_15_49,males_50_64,males_5_9,males_65_plus,pop_0_4,pop_0_9,pop_10_14,pop_15_49,pop_50_64,pop_5_9,pop_65_plus,population,pregnancies,rwi,time_delta_constructed_sites_all_education_facilities_fixed,time_delta_constructed_sites_all_health_facilities_optimal,time_delta_constructed_sites_health_centers_optimal,time_delta_constructed_sites_health_posts_optimal,time_delta_constructed_sites_major_hospitals_optimal,time_delta_constructed_sites_primary_schools_fixed,time_delta_constructed_sites_secondary_schools_fixed,time_delta_constructed_sites_semi_dense_urban_optimal,time_delta_no_sites_all_education_facilities_fixed,time_delta_no_sites_all_health_facilities_optimal,time_delta_no_sites_health_centers_optimal,time_delta_no_sites_health_posts_optimal,time_delta_no_sites_major_hospitals_optimal,time_delta_no_sites_primary_schools_fixed,time_delta_no_sites_secondary_schools_fixed,time_delta_no_sites_semi_dense_urban_optimal,travel_time_all_education_facilities_fixed,travel_time_all_health_facilities_optimal,travel_time_constructed_sites_all_education_facilities_fixed,travel_time_constructed_sites_all_health_facilities_optimal,travel_time_constructed_sites_health_centers_optimal,travel_time_constructed_sites_health_posts_optimal,travel_time_constructed_sites_major_hospitals_optimal,travel_time_constructed_sites_primary_schools_fixed,travel_time_constructed_sites_secondary_schools_fixed,travel_time_constructed_sites_semi_dense_urban_optimal,travel_time_health_centers_optimal,travel_time_health_posts_optimal,travel_time_major_hospitals_optimal,travel_time_no_sites_all_education_facilities_fixed,travel_time_no_sites_all_health_facilities_optimal,travel_time_no_sites_health_centers_optimal,travel_time_no_sites_health_posts_optimal,travel_time_no_sites_major_hospitals_optimal,travel_time_no_sites_primary_schools_fixed,travel_time_no_sites_secondary_schools_fixed,travel_time_no_sites_semi_dense_urban_optimal,travel_time_primary_schools_fixed,travel_time_secondary_schools_fixed,travel_time_semi_dense_urban_optimal,underweight,x_y
0,10_1897,0.2158,11.537741,0.532922,1.102213,0.570825,1.990846,0.334852,0.569291,0.116536,POINT (30.43849 -1.05409),14.53916,0.523792,1.12397,0.606819,2.048758,0.421264,0.600178,0.120037,1.056714,2.226183,1.177644,4.039604,0.756116,1.169469,0.236573,8.436119,0.3056,-0.746829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,81.0,82.0,81.0,82.0,82.0,253.0,870.0,81.0,81.0,507.0,82.0,253.0,870.0,81.0,82.0,82.0,253.0,870.0,81.0,81.0,507.0,81.0,81.0,507.0,0.780426,30.4384884_-1.0540862
1,10_1898,0.203684,10.892005,0.503002,1.04033,0.538776,1.879072,0.316052,0.537329,0.109993,POINT (30.43917 -1.05417),13.723303,0.494384,1.060866,0.57275,1.933732,0.397613,0.566482,0.113297,0.997386,2.101196,1.111526,3.812804,0.713665,1.10381,0.22329,7.962481,0.288443,-0.683313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0,80.0,79.0,80.0,80.0,251.0,868.0,79.0,79.0,505.0,80.0,251.0,868.0,79.0,80.0,80.0,251.0,868.0,79.0,79.0,505.0,79.0,79.0,505.0,0.736788,30.439166560383864_-1.0541664609655237
2,10_1899,0.373898,20.021989,0.923348,1.909709,0.989018,3.449366,0.580169,0.986361,0.201912,POINT (30.44000 -1.05417),25.213717,0.907529,1.947405,1.051382,3.549704,0.729888,1.039876,0.207977,1.830877,3.857114,2.040401,6.999069,1.310057,2.026237,0.409889,14.616529,0.529487,-1.231552,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,77.0,78.0,77.0,78.0,78.0,249.0,866.0,77.0,77.0,503.0,78.0,249.0,866.0,77.0,78.0,78.0,249.0,866.0,77.0,77.0,503.0,77.0,77.0,503.0,1.355002,30.439999893713928_-1.0541664609655237
3,10_1900,0.579211,31.039584,1.430372,2.958358,1.532103,5.343463,0.898748,1.527986,0.312784,POINT (30.44113 -1.05446),39.071786,1.405866,3.016753,1.628711,5.498899,1.13068,1.610887,0.32

In [61]:
def aggregate_lists(x):
    return '&'.join(x)
    
def male_educational_attainment_weighted_average(x):
    # Calculate the weighted average using np.average and weights based on sum of males
    total_males = travel_time_merged.loc[x.index, ['males_15_49', 'males_50_64', 'males_65_plus']].sum(axis=1)
    total_weight = total_males.sum()
    if total_weight == 0:
        # If the total weight is zero, return a default value (e.g., 0) or handle it as per your requirement.
        return 0
    non_zero_mask = (total_males > 0) & (~x.isnull()) # Mask to avoid dividing by zero
    if non_zero_mask.any():
        weighted_avg = np.average(x[non_zero_mask], weights=total_males[non_zero_mask])
        return weighted_avg
    else:
        weighted_avg = np.nan
        return weighted_avg
def female_educational_attainment_weighted_average(x):
    # Calculate the weighted average using np.average and weights based on sum of females
    total_females = travel_time_merged.loc[x.index, ['females_15_49', 'females_50_64', 'females_65_plus']].sum(axis=1)
    total_weight = total_females.sum()
    
    if total_weight == 0:
        # If the total weight is zero, return a default value (e.g., 0) or handle it as per your requirement.
        return 0
    
    non_zero_mask = (total_females > 0) & (~x.isnull())  # Mask to avoid dividing by zero
    if non_zero_mask.any():
        weighted_avg = np.average(x[non_zero_mask], weights=total_females[non_zero_mask])
        return weighted_avg
    else:
        weighted_avg = np.nan
        return weighted_avg

def weighted_average_function(values, weights):
    if weights.sum() == 0:
        return np.nan
    if weights.sum() == np.nan:
        return np.nan
    if weights.sum() == np.inf:
        return np.nan
    else:
        return np.average(values, weights=weights)


joining_rules = {
'row_col': aggregate_lists,
'x_y': aggregate_lists,
'female_educational_attainment_mean': 'sum', 
'females_0_4': 'sum', 
'females_0_9': 'sum', 
'females_10_14': 'sum', 
'females_15_49': 'sum', 
'females_50_64': 'sum', 
'females_5_9': 'sum', 
'females_65_plus': 'sum', 
'male_educational_attainment_mean': 'sum',
'males_0_4': 'sum', 
'males_0_9': 'sum', 
'males_10_14': 'sum', 
'males_15_49': 'sum', 
'males_50_64': 'sum', 
'males_5_9': 'sum', 
'males_65_plus': 'sum', 
'pop_0_4': 'sum', 
'pop_0_9': 'sum', 
'pop_10_14': 'sum', 
'pop_15_49': 'sum', 
'pop_50_64': 'sum', 
'pop_5_9': 'sum', 
'pop_65_plus': 'sum', 
'population': 'sum', 
'pregnancies': 'sum',
'births': 'sum', 
'rwi': 'sum',
'underweight': 'sum', 
'time_delta_constructed_sites_all_education_facilities_fixed': 'mean', 
'time_delta_constructed_sites_all_health_facilities_optimal': 'mean', 
'time_delta_constructed_sites_health_centers_optimal': 'mean', 
'time_delta_constructed_sites_health_posts_optimal': 'mean', 
'time_delta_constructed_sites_major_hospitals_optimal': 'mean', 
'time_delta_constructed_sites_primary_schools_fixed': 'mean', 
'time_delta_constructed_sites_secondary_schools_fixed': 'mean', 
'time_delta_constructed_sites_semi_dense_urban_optimal': 'mean', 
'time_delta_no_sites_all_education_facilities_fixed': 'mean', 
'time_delta_no_sites_all_health_facilities_optimal': 'mean', 
'time_delta_no_sites_health_centers_optimal': 'mean', 
'time_delta_no_sites_health_posts_optimal': 'mean', 
'time_delta_no_sites_major_hospitals_optimal': 'mean', 
'time_delta_no_sites_primary_schools_fixed': 'mean', 
'time_delta_no_sites_secondary_schools_fixed': 'mean', 
'time_delta_no_sites_semi_dense_urban_optimal': 'mean', 
'travel_time_all_education_facilities_fixed': 'mean', 
'travel_time_all_health_facilities_optimal': 'mean', 
'travel_time_constructed_sites_all_education_facilities_fixed': 'mean', 
'travel_time_constructed_sites_all_health_facilities_optimal': 'mean', 
'travel_time_constructed_sites_health_centers_optimal': 'mean', 
'travel_time_constructed_sites_health_posts_optimal': 'mean', 
'travel_time_constructed_sites_major_hospitals_optimal': 'mean', 
'travel_time_constructed_sites_primary_schools_fixed': 'mean', 
'travel_time_constructed_sites_secondary_schools_fixed': 'mean', 
'travel_time_constructed_sites_semi_dense_urban_optimal': 'mean', 
'travel_time_health_centers_optimal': 'mean', 
'travel_time_health_posts_optimal': 'mean', 
'travel_time_major_hospitals_optimal': 'mean', 
'travel_time_no_sites_all_education_facilities_fixed': 'mean', 
'travel_time_no_sites_all_health_facilities_optimal': 'mean', 
'travel_time_no_sites_health_centers_optimal': 'mean', 
'travel_time_no_sites_health_posts_optimal': 'mean', 
'travel_time_no_sites_major_hospitals_optimal': 'mean', 
'travel_time_no_sites_primary_schools_fixed': 'mean', 
'travel_time_no_sites_secondary_schools_fixed': 'mean', 
'travel_time_no_sites_semi_dense_urban_optimal': 'mean', 
'travel_time_primary_schools_fixed': 'mean', 
'travel_time_secondary_schools_fixed': 'mean', 
'travel_time_semi_dense_urban_optimal': 'mean',
}

In [79]:
# This is causing all values in a row to be the same.. need to fix
def make_hexagons(resolution, write_to_file):
    # make travel_time_merged_hex a geopandas dataframe that matches travel_time_merged
    travel_time_merged_hex = travel_time_merged.copy()
    travel_time_merged_hex = gpd.GeoDataFrame(travel_time_merged_hex, geometry='geometry')
    travel_time_merged_hex.crs = "EPSG:4326"

    travel_time_merged_hex['h3-index'] = None

    for idx, row in travel_time_merged_hex.iterrows():
        try:
            lat = row['geometry'].y
            lon = row['geometry'].x
            travel_time_merged_hex.at[idx, 'h3-index'] = h3.geo_to_h3(lat, lon, resolution)
        except:
            pass

    # merge all rows that match on h3-index following joining rules
    travel_time_merged_hex = travel_time_merged_hex.groupby('h3-index').agg(joining_rules)
    travel_time_merged_hex = travel_time_merged_hex.reset_index()
    # drop any rows without a h3-index
    travel_time_merged_hex = travel_time_merged_hex.dropna(subset=['h3-index'])
    # convert h3-index to polygon
    travel_time_merged_hex['geometry'] = travel_time_merged_hex['h3-index'].apply(lambda x: h3.h3_to_geo_boundary(x, True))
    # make geometry column a polygon
    travel_time_merged_hex['geometry'] = travel_time_merged_hex['geometry'].apply(lambda x: Polygon(x))
    # convert travel_time_merged_hex to geodataframe
    travel_time_merged_hex = gpd.GeoDataFrame(travel_time_merged_hex, geometry='geometry')
    travel_time_merged_hex.crs = "EPSG:4326"
    if write_to_file == True:
        travel_time_merged_hex.to_file("./synced-data/rwa_travel_time_hex-"+str(resolution)+".geojson", driver='GeoJSON', na='null')
    return travel_time_merged_hex

In [92]:
hex8 = make_hexagons(8, False)

In [135]:
# Normalizing the weighted population columns
hex8['rwi'] = hex8.apply(lambda row: row['rwi'] / row['population'] if row['population'] != 0 else np.nan, axis=1)
hex8['underweight'] = hex8.apply(lambda row: row['underweight'] / row['population'] if row['population'] != 0 else np.nan, axis=1)
hex8['male_educational_attainment_mean'] = hex8.apply(lambda row: row['male_educational_attainment_mean'] / (row['males_15_49'] + row['males_50_64'] + row['males_65_plus']) if (row['males_15_49'] + row['males_50_64'] + row['males_65_plus']) != 0 else np.nan, axis=1)
hex8['female_educational_attainment_mean'] = hex8.apply(lambda row: row['female_educational_attainment_mean'] / (row['females_15_49'] + row['females_50_64'] + row['females_65_plus']) if (row['females_15_49'] + row['females_50_64'] + row['females_65_plus']) != 0 else np.nan, axis=1)

In [136]:
# Checking to see normalizaton worked. These should be just above 8
print(hex8['female_educational_attainment_mean'].max())
print(hex8['male_educational_attainment_mean'].max())

8.658925975996027
8.966146346597323


In [169]:
# should do this after merging with hexagons so  that we don't lose precision
# change datatypes of columns
print(hex8.dtypes)
int_list = ['time_delta_constructed_sites_all_education_facilities_fixed', 'time_delta_constructed_sites_all_health_facilities_optimal', 'time_delta_constructed_sites_health_centers_optimal', 'time_delta_constructed_sites_health_posts_optimal', 'time_delta_constructed_sites_major_hospitals_optimal', 'time_delta_constructed_sites_primary_schools_fixed', 'time_delta_constructed_sites_secondary_schools_fixed', 'time_delta_constructed_sites_semi_dense_urban_optimal', 'time_delta_no_sites_all_education_facilities_fixed', 'time_delta_no_sites_all_health_facilities_optimal', 'time_delta_no_sites_health_centers_optimal', 'time_delta_no_sites_health_posts_optimal', 'time_delta_no_sites_major_hospitals_optimal', 'time_delta_no_sites_primary_schools_fixed', 'time_delta_no_sites_secondary_schools_fixed', 'time_delta_no_sites_semi_dense_urban_optimal', 'travel_time_all_education_facilities_fixed', 'travel_time_all_health_facilities_optimal', 'travel_time_constructed_sites_all_education_facilities_fixed', 'travel_time_constructed_sites_all_health_facilities_optimal', 'travel_time_constructed_sites_health_centers_optimal', 'travel_time_constructed_sites_health_posts_optimal', 'travel_time_constructed_sites_major_hospitals_optimal', 'travel_time_constructed_sites_primary_schools_fixed', 'travel_time_constructed_sites_secondary_schools_fixed', 'travel_time_constructed_sites_semi_dense_urban_optimal', 'travel_time_health_centers_optimal', 'travel_time_health_posts_optimal', 'travel_time_major_hospitals_optimal', 'travel_time_no_sites_all_education_facilities_fixed', 'travel_time_no_sites_all_health_facilities_optimal', 'travel_time_no_sites_health_centers_optimal', 'travel_time_no_sites_health_posts_optimal', 'travel_time_no_sites_major_hospitals_optimal', 'travel_time_no_sites_primary_schools_fixed', 'travel_time_no_sites_secondary_schools_fixed', 'travel_time_no_sites_semi_dense_urban_optimal', 'travel_time_primary_schools_fixed', 'travel_time_secondary_schools_fixed', 'travel_time_semi_dense_urban_optimal', 'males_0_4', 'males_0_9', 'males_10_14', 'males_15_49', 'males_50_64', 'males_5_9', 'males_65_plus', 'pop_0_4', 'pop_0_9', 'pop_10_14', 'pop_15_49', 'pop_50_64', 'pop_5_9', 'pop_65_plus', 'population', 'females_0_4', 'females_0_9', 'females_10_14', 'females_15_49', 'females_50_64', 'females_5_9', 'females_65_plus', 'births', 'pregnancies', ]
float_list = ['rwi', 'underweight', 'female_educational_attainment_mean', 'male_educational_attainment_mean']
string_list = ['x_y', 'row_col']
for col in int_list:
        hex8[col] = hex8[col].astype(int, errors='ignore')
for col in float_list:
    hex8[col] = pd.to_numeric(hex8[col], errors='coerce', downcast='float')
    hex8[col] = hex8[col].round(4)

print(hex8.dtypes)

h3-index                                           object
row_col                                            object
female_educational_attainment_mean                float32
females_0_4                                         int64
females_0_9                                         int64
                                                   ...   
travel_time_no_sites_semi_dense_urban_optimal       int64
travel_time_primary_schools_fixed                  object
travel_time_secondary_schools_fixed                object
travel_time_semi_dense_urban_optimal                int64
geometry                                         geometry
Length: 71, dtype: object
h3-index                                           object
row_col                                            object
female_educational_attainment_mean                float32
females_0_4                                         int64
females_0_9                                         int64
                                              

In [138]:
# drop x_y from hex8. We won't use this in the app
hex8 = hex8.drop(['x_y'], axis=1)

In [173]:
hex8.to_file("./synced-data/rwa_travel_time_hex-8.geojson", driver='GeoJSON', na='null', index=False)

# Creating a hex to subregion lookup table

In [112]:
# create a dataframe with just h3-index and row_col
lookup = hex8[['h3-index', 'row_col']]

In [114]:
# create a new dataframe called lookup_full that has two columns: h3-index and row_col. row_col is the values from each row split on & with the h3-index repeated for each row_col value
lookup_full = pd.DataFrame(columns=['h3-index', 'row_col'])
for idx, row in lookup.iterrows():
    row_col_list = row['row_col'].split('&')
    for row_col in row_col_list:
        lookup_full = lookup_full.append({'h3-index': row['h3-index'], 'row_col': row_col}, ignore_index=True)


AttributeError: 'DataFrame' object has no attribute 'append'

In [115]:

# Assuming you have a DataFrame called 'lookup' with columns 'h3-index' and 'row_col'

# create a list to store dictionaries for each row in the new DataFrame
data = []

for idx, row in lookup.iterrows():
    row_col_list = row['row_col'].split('&')
    for row_col in row_col_list:
        data.append({'h3-index': row['h3-index'], 'row_col': row_col})

# create the new DataFrame 'lookup_full' using the list of dictionaries
lookup_full = pd.DataFrame(data)

# If needed, you can set the data types of the columns explicitly, for example:
# lookup_full = pd.DataFrame(data, dtype={'h3-index': int, 'row_col': str})


In [116]:
lookup_full

,h3-index,row_col
0,886ad80001fffff,1334_1576
1,886ad80001fffff,1335_1573
2,886ad80001fffff,1335_1574
3,886ad80001fffff,1335_1575
4,886ad80001fffff,1335_1576
...,...,...
1150599,886adeb76dfffff,760_600
1150600,886adeb76dfffff,760_601
1150601,886adeb76dfffff,760_602
1150602,886adeb893fffff,974_516


In [118]:
# how many unique values of h3-index are there
len(lookup_full['h3-index'].unique())

26138

In [124]:
# write lookup_full to json file
lookup_full.to_json('./synced-data/subregion-to-hex-index.json', orient='records')

In [126]:
# write lookup_full to csv file
lookup_full.to_csv('./synced-data/subregion-to-hex-index.csv', index=False)

# Create a dataMap dictionary for the front end application

In [150]:
hex8 = gpd.read_file('../public/rwa_travel_time_hex-8.geojson')
hex8

KeyboardInterrupt: 

In [8]:
# create a dictionary that is the  names of the column names in hex8 and the values are an empty list
hex8_dict = {}
for col in hex8_new.columns:
    hex8_dict[col] = []
hex8_dict


{'h3-index': [],
 'row_col': [],
 'female_educational_attainment_mean': [],
 'females_0_4': [],
 'females_0_9': [],
 'females_10_14': [],
 'females_15_49': [],
 'females_50_64': [],
 'females_5_9': [],
 'females_65_plus': [],
 'male_educational_attainment_mean': [],
 'males_0_4': [],
 'males_0_9': [],
 'males_10_14': [],
 'males_15_49': [],
 'males_50_64': [],
 'males_5_9': [],
 'males_65_plus': [],
 'pop_0_4': [],
 'pop_0_9': [],
 'pop_10_14': [],
 'pop_15_49': [],
 'pop_50_64': [],
 'pop_5_9': [],
 'pop_65_plus': [],
 'population': [],
 'pregnancies': [],
 'births': [],
 'rwi': [],
 'underweight': [],
 'time_delta_constructed_sites_all_education_facilities_fixed': [],
 'time_delta_constructed_sites_all_health_facilities_optimal': [],
 'time_delta_constructed_sites_health_centers_optimal': [],
 'time_delta_constructed_sites_health_posts_optimal': [],
 'time_delta_constructed_sites_major_hospitals_optimal': [],
 'time_delta_constructed_sites_primary_schools_fixed': [],
 'time_delta_con

In [183]:
data_map = {
    "key": ["Menu Options", "hover panel text", "color scheme"],
    "h3-index": [["N/A"], ["H3 Index", ""], "rainbow"],
    "row_col": [["N/A"], ["Row/Col", ""], "rainbow"],
    "female_educational_attainment_mean": [["Demographics", "Female Education", "N/A"], ["Avg. Female Education", "years"], "rdylgn"],
    "females_0_4": [["Population", "Female 0-4", "N/A"], ["Population", "females"], "magma"],
    "females_0_9": [["Population", "Female 0-9", "N/A"], ["Population", "females"], "magma"],
    "females_10_14": [["Population", "Female 10-14", "N/A"], ["Population", "females"], "magma"],
    "females_15_49": [["Population", "Female 15-49", "N/A"], ["Population", "females"], "magma"],
    "females_50_64": [["Population", "Female 50-64", "N/A"], ["Population", "females"], "magma"],
    "females_5_9": [["Population", "Female 5-9", "N/A"], ["Population", "females"], "magma"],
    "females_65_plus": [["Population", "Female 65+", "N/A"], ["Population", "females"], "magma"],
    "male_educational_attainment_mean": [["Demographics", "Male Education", "N/A"], ["Avg. Male Education", "years"], "rdylgn"],
    "males_0_4": [["Population", "Male 0-4", "N/A"], ["Population", "males"], "magma"],
    "males_0_9": [["Population", "Male 0-9", "N/A"], ["Population", "males"], "magma"],
    "males_10_14": [["Population", "Male 10-14", "N/A"], ["Population", "males"], "magma"],
    "males_15_49": [["Population", "Male 15-49", "N/A"], ["Population", "males"], "magma"],
    "males_50_64": [["Population", "Male 50-64", "N/A"], ["Population", "males"], "magma"],
    "males_5_9": [["Population", "Male 5-9", "N/A"], ["Population", "males"], "magma"],
    "males_65_plus": [["Population", "Male 65+", "N/A"], ["Population", "males"], "magma"],
    "pop_0_4": [["Population", "Children 0-4", "N/A"], ["Population", "people"], "magma"],
    "pop_0_9": [["Population", "Children 0-9", "N/A"], ["Population", "people"], "magma"],
    "pop_10_14": [["Population", "Youth 10-14", "N/A"], ["Population", "people"], "magma"],
    "pop_15_49": [["Population", "People 15-49", "N/A"], ["Population", "people"], "magma"],
    "pop_50_64": [["Population", "People 50-64", "N/A"], ["Population", "people"], "magma"],
    "pop_5_9": [["Population", "Children 5-9", "N/A"], ["Population", "people"], "magma"],
    "pop_65_plus": [["Population", "People 65+", "N/A"], ["Population", "people"], "magma"],
    "population": [["Population", "All", "N/A"], ["Population", "people"], "magma"],
    "pregnancies": [["Demographics", "Pregnancies", "N/A"], ["Pregnancies"], "magma"],
    "births": [["Demographics", "Births", "N/A"], ["Births"], "magma"],
    "rwi": [["Demographics", "Relative Wealth Index", "N/A"], ["Relative Wealth Index"], "rdylgn"],
    "underweight": [["Demographics", "Underweight", "N/A"], ["Underweight"], "rdylgn"],
    "time_delta_constructed_sites_all_education_facilities_fixed": [["Impact", "Current Impact", "School"], "magma"],
    "time_delta_constructed_sites_all_health_facilities_optimal": [["Impact", "Current Impact", "Healthcare"], "magma"],
    "time_delta_constructed_sites_health_centers_optimal": [["Impact", "Current Impact", "Health Center"], "magma"],
    "time_delta_constructed_sites_health_posts_optimal": [["Impact", "Current Impact", "Health Post"], "magma"],
    "time_delta_constructed_sites_major_hospitals_optimal": [["Impact", "Current Impact", "Major Hospital"], "magma"],
    "time_delta_constructed_sites_primary_schools_fixed": [["Impact", "Current Impact", "Primary School"], "magma"],
    "time_delta_constructed_sites_secondary_schools_fixed": [["Impact", "Current Impact", "Secondary School"], "magma"],
    "time_delta_constructed_sites_semi_dense_urban_optimal": [["Impact", "Current Impact", "Semi Dense Urban"], "magma"],
    "time_delta_no_sites_all_education_facilities_fixed": [["Impact", "Potential Impact", "School"], "magma"],
    "time_delta_no_sites_all_health_facilities_optimal": [["Impact", "Potential Impact", "Healthcare"], "magma"],
    "time_delta_no_sites_health_centers_optimal": [["Impact", "Potential Impact", "Health Center"], "magma"],
    "time_delta_no_sites_health_posts_optimal": [["Impact", "Potential Impact", "Health Post"], "magma"],
    "time_delta_no_sites_major_hospitals_optimal": [["Impact", "Potential Impact", "Major Hospital"], "magma"],
    "time_delta_no_sites_primary_schools_fixed": [["Impact", "Potential Impact", "Primary School"], "magma"],
    "time_delta_no_sites_secondary_schools_fixed": [["Impact", "Potential Impact", "Secondary School"], "magma"],
    "time_delta_no_sites_semi_dense_urban_optimal": [["Impact", "Potential Impact", "Market"], "magma"],
    "travel_time_all_education_facilities_fixed": [["Travel Time", "With All Bridges", "To School"], "rdylgn"],
    "travel_time_all_health_facilities_optimal": [["Travel Time", "With All Bridges", "To Healthcare"], "rdylgn"],
    "travel_time_constructed_sites_all_education_facilities_fixed": [["Travel Time", " With Constructed Bridges", "To School"], "rdylgn"],
    "travel_time_constructed_sites_all_health_facilities_optimal": [["Travel Time", "With Constructed Bridges", "To Healthcare"], "rdylgn"],
    "travel_time_constructed_sites_health_centers_optimal": [["Travel Time", "With Constructed Bridges", "To Health Center"], "rdylgn"],
    "travel_time_constructed_sites_health_posts_optimal": [["Travel Time", "With Constructed Bridges", "To Health Post"], "rdylgn"],
    "travel_time_constructed_sites_major_hospitals_optimal": [["Travel Time", "With Constructed Bridges", "To Hospital"], "rdylgn"],
    "travel_time_constructed_sites_primary_schools_fixed": [["Travel Time", "With Constructed Bridges", "To Primary School"], "rdylgn"],
    "travel_time_constructed_sites_secondary_schools_fixed": [["Travel Time", "With Constructed Bridges", "To Secondary School"], "rdylgn"],
    "travel_time_constructed_sites_semi_dense_urban_optimal": [["Travel Time", "With Constructed Bridges", "To Market"], "rdylgn"],
    "travel_time_health_centers_optimal": [["Travel Time", "With Bridges", "To Health Center"], "rdylgn"],
    "travel_time_health_posts_optimal": [["Travel Time", "With Bridges", "To Health Post"], "rdylgn"],
    "travel_time_major_hospitals_optimal": [["Travel Time", "With Bridges", "To Hospital"], "rdylgn"],
    "travel_time_no_sites_all_education_facilities_fixed": [["Travel Time", "Without Bridges", "To School"], "rdylgn"],
    "travel_time_no_sites_all_health_facilities_optimal": [["Travel Time", "Without Bridges", "To Healthcare"], "rdylgn"],
    "travel_time_no_sites_health_centers_optimal": [["Travel Time", "Without Bridges", "To Health Center"], "rdylgn"],
    "travel_time_no_sites_health_posts_optimal": [["Travel Time", "Without Bridges", "To Health Post"], "rdylgn"],
    "travel_time_no_sites_major_hospitals_optimal": [["Travel Time", "Without Bridges", "To Hospital"], "rdylgn"],
    "travel_time_no_sites_primary_schools_fixed": [["Travel Time", "Without Bridges", " To Primary School"], "rdylgn"],
    "travel_time_no_sites_secondary_schools_fixed": [["Travel Time", "Without Bridges", " To Secondary School"], "rdylgn"],
    "travel_time_no_sites_semi_dense_urban_optimal": [["Travel Time", "Without Bridges", "To Market"], "rdylgn"],
    "travel_time_primary_schools_fixed": [["Travel Time", "With All Bridges", "To Primary School"], "rdylgn"],
    "travel_time_secondary_schools_fixed": [["Travel Time", "With All Bridges", "To Secondary School"], "rdylgn"],
    "travel_time_semi_dense_urban_optimal": [["Travel Time", "With All Bridges", "To Market"], "rdylgn"],
    'all_education_potential_impact': [["Impact", "Potential Impact", "Education"], "magma"],
    'all_education_current_impact': [["Impact", "Current Impact", "Education"], "magma"],
    'all_health_potential_impact': [],
    'all_health_current_impact': [],
    'health_centers_potential_impact': [],
    'health_centers_current_impact': [],
    'health_posts_potential_impact': [],
    'health_posts_current_impact': [],
    'major_hospitals_potential_impact': [],
    'major_hospitals_current_impact': [],
    'primary_schools_potential_impact': [],
    'primary_schools_current_impact': [],
    'secondary_schools_potential_impact': [],
    'secondary_schools_current_impact': [],
    'semi_dense_urban_potential_impact': [],
    'semi_dense_urban_current_impact': [],
    'total_potential_impact': [],
    'total_current_impact': [],
    'total_school_current_impact': [],
    'total_school_potential_impact': [],
    'total_health_current_impact': [],
    'total_health_potential_impact': [],
}
for key, value in data_map.items():
    if key in ["key", "h3-index", "row_col"]:
        print("skipping " + key)
    else:
        key_max = hex8[key].max()
        key_min = hex8[key].min()
        key_mean = hex8[key].mean()
        key_median = hex8[key].median()
        key_98 = hex8[key].quantile(.98)
        key_99 = hex8[key].quantile(.99)
        key_2 = hex8[key].quantile(.02)
        key_1 = hex8[key].quantile(.01)
        key_quantiles = [hex8[key].quantile(0), hex8[key].quantile(0.1), hex8[key].quantile(0.2), hex8[key].quantile(0.3), hex8[key].quantile(
            0.4), hex8[key].quantile(0.5), hex8[key].quantile(0.6), hex8[key].quantile(0.7), hex8[key].quantile(0.8), hex8[key].quantile(0.9), hex8[key].quantile(1)]
        style_stops_99 = [hex8[key].quantile(.01), ((hex8[key].quantile(.99)-hex8[key].quantile(.01))*0.25)+hex8[key].quantile(.01), ((hex8[key].quantile(.99)-hex8[key].quantile(.01))
                                                                                                                                      * 0.5)+hex8[key].quantile(.01), ((hex8[key].quantile(.99)-hex8[key].quantile(.01))*0.75)+hex8[key].quantile(.01), hex8[key].quantile(.99)]
        style_stops_98 = [hex8[key].quantile(.02), ((hex8[key].quantile(.98)-hex8[key].quantile(.02))*0.25)+hex8[key].quantile(.02), ((hex8[key].quantile(.98)-hex8[key].quantile(.02))
                                                                                                                                      * 0.5)+hex8[key].quantile(.02), ((hex8[key].quantile(.98)-hex8[key].quantile(.02))*0.75)+hex8[key].quantile(.02), hex8[key].quantile(.98)]
        style_stops_maxmin = [hex8[key].min(), ((hex8[key].max()-hex8[key].min())*0.25)+hex8[key].min(), ((hex8[key].max(
        )-hex8[key].min())*0.5)+hex8[key].min(), ((hex8[key].max()-hex8[key].min())*0.75)+hex8[key].min(), hex8[key].max()]

        # Initialize an empty list if key is not in data_map
        if key not in data_map:
            data_map[key] = []

        # Append the statistics dictionary to the list
        data_map[key].append({'max': key_max, 'min': key_min, 'mean': key_mean, 'median': key_median, '98_percentile': key_98, '99_percentile': key_99, '2_percentile': key_2,
                             '1_percentile': key_1, 'quantiles': key_quantiles, 'style_stops_99': style_stops_99, 'style_stops_98': style_stops_98, 'style_stops_maxmin': style_stops_maxmin})

skipping key
skipping h3-index
skipping row_col


In [184]:
data_map

{'key': ['Menu Options', 'hover panel text', 'color scheme'],
 'h3-index': [['N/A'], ['H3 Index', ''], 'rainbow'],
 'row_col': [['N/A'], ['Row/Col', ''], 'rainbow'],
 'female_educational_attainment_mean': [['Demographics',
   'Female Education',
   'N/A'],
  ['Avg. Female Education', 'years'],
  'rdylgn',
  {'max': 8.6589,
   'min': 3.4551,
   'mean': 4.8228326,
   'median': 4.7054,
   '98_percentile': 6.68306,
   '99_percentile': 7.185144,
   '2_percentile': 3.999912,
   '1_percentile': 3.921612,
   'quantiles': [3.4551,
    4.26988,
    4.4051,
    4.5131,
    4.6083,
    4.7054,
    4.80628,
    4.9392,
    5.11134,
    5.48352,
    8.6589],
   'style_stops_99': [3.921612,
    4.737495005130768,
    5.553377985954285,
    6.3692609667778015,
    7.185144],
   'style_stops_98': [3.999912,
    4.670699059963226,
    5.341486096382141,
    6.012273132801056,
    6.68306],
   'style_stops_maxmin': [3.4551,
    4.756050109863281,
    6.057000160217285,
    7.357950210571289,
    8.6589]}

In [193]:
# create a dictionary that is the  names of the column names in hex8 and the values are an empty list
style_hex_dict = {}
for col in hex8.columns: 
  style_hex_dict[col] = { 'fill-color': [ "'interpolate', ['linear'], ['get', '"+col+"'], dataMap['"+col+"'][3]['style_stops_99'][0], palettes[dataMap['"+col+"'][2]][0], dataMap['"+col+"'][3]['style_stops_99'][1], palettes[dataMap['"+col+"'][2]][1], dataMap['"+col+"'][3]['style_stops_99'][2], palettes[dataMap['"+col+"'][2]][2], dataMap['"+col+"'][3]['style_stops_99'][3], palettes[dataMap['"+col+"'][2]][3], dataMap['"+col+"'][3]['style_stops_99'][4], palettes[dataMap['"+col+"'][2]][4]"]}

In [195]:
style_hex_dict

{'h3-index': {'fill-color': ["'interpolate', ['linear'], ['get', 'h3-index'], dataMap['h3-index'][3]['style_stops_99'][0], palettes[dataMap['h3-index'][2]][0], dataMap['h3-index'][3]['style_stops_99'][1], palettes[dataMap['h3-index'][2]][1], dataMap['h3-index'][3]['style_stops_99'][2], palettes[dataMap['h3-index'][2]][2], dataMap['h3-index'][3]['style_stops_99'][3], palettes[dataMap['h3-index'][2]][3], dataMap['h3-index'][3]['style_stops_99'][4], palettes[dataMap['h3-index'][2]][4]"]},
 'row_col': {'fill-color': ["'interpolate', ['linear'], ['get', 'row_col'], dataMap['row_col'][3]['style_stops_99'][0], palettes[dataMap['row_col'][2]][0], dataMap['row_col'][3]['style_stops_99'][1], palettes[dataMap['row_col'][2]][1], dataMap['row_col'][3]['style_stops_99'][2], palettes[dataMap['row_col'][2]][2], dataMap['row_col'][3]['style_stops_99'][3], palettes[dataMap['row_col'][2]][3], dataMap['row_col'][3]['style_stops_99'][4], palettes[dataMap['row_col'][2]][4]"]},
 'female_educational_attainmen

# recreating dictionary with updated data

In [3]:
hex8_new = gpd.read_file('https://fikamap-web-app.s3.us-west-1.amazonaws.com/data/rwa_travel_time_hex-8.geojson')


In [7]:
create_style_dictionary(hex8_new)

{'h3-index': {'fill-color': ["'interpolate', ['linear'], ['get', 'h3-index'], dataMap['h3-index'][3]['style_stops_99'][0], palettes[dataMap['h3-index'][2]][0], dataMap['h3-index'][3]['style_stops_99'][1], palettes[dataMap['h3-index'][2]][1], dataMap['h3-index'][3]['style_stops_99'][2], palettes[dataMap['h3-index'][2]][2], dataMap['h3-index'][3]['style_stops_99'][3], palettes[dataMap['h3-index'][2]][3], dataMap['h3-index'][3]['style_stops_99'][4], palettes[dataMap['h3-index'][2]][4]"]},
 'row_col': {'fill-color': ["'interpolate', ['linear'], ['get', 'row_col'], dataMap['row_col'][3]['style_stops_99'][0], palettes[dataMap['row_col'][2]][0], dataMap['row_col'][3]['style_stops_99'][1], palettes[dataMap['row_col'][2]][1], dataMap['row_col'][3]['style_stops_99'][2], palettes[dataMap['row_col'][2]][2], dataMap['row_col'][3]['style_stops_99'][3], palettes[dataMap['row_col'][2]][3], dataMap['row_col'][3]['style_stops_99'][4], palettes[dataMap['row_col'][2]][4]"]},
 'female_educational_attainmen